In [ ]:
import pandas as pd
df= pd.read_csv("/content/Final_EnhancedGmail.csv")

In [ ]:
df=df.iloc[11:21]

In [ ]:
df.columns

Index(['Genre', 'Reports', 'Age', 'Year', 'Count', 'Gender', 'Nationality',
       'Resolution', 'Student Name', 'Ticket ID', 'Student_ID', 'GPA',
       'priority', 'status', 'report_time', 'report_hour', 'response_time',
       'response_days', 'Gmail'],
      dtype='object')

In [ ]:
from transformers import pipeline
import torch
from tqdm import tqdm

# Initialize model with GPU if available
device = 0 if torch.cuda.is_available() else -1
feedback_generator = pipeline(
    "text2text-generation",
    model="google/flan-t5-large",  # 780M parameters
    device="cuda" if torch.cuda.is_available() else "cpu",
    torch_dtype=torch.float16  # Cuts memory by 50%
)

def generate_feedback(issue, solution):
    prompt = f"""Provide constructive feedback on this solution for the student's issue based on their report:

    Student's Issue: {issue}
    Proposed Solution: {solution}

    Professional Feedback:"""

    feedback = feedback_generator(
        prompt,
        max_length=200,
        do_sample=True,
        temperature=0.7
    )
    return feedback[0]['generated_text']

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
df['Feedback'] = [generate_feedback(row['Reports'], row['Resolution'])
                 for _, row in tqdm(df.iterrows(), total=len(df), desc="Generating Feedback")]

Generating Feedback:   6%|▌         | 566/10000 [1:33:36<23:30:39,  8.97s/it]

KeyboardInterrupt: 

In [ ]:
# Initialize the pipeline (corrected spelling)
recommendation_generator = pipeline(
    "text2text-generation",
    model="google/flan-t5-large",  # 780M parameters
    device="cuda" if torch.cuda.is_available() else "cpu",
    torch_dtype=torch.float16  # Cuts memory by 50%
)

def generate_recommendation(issue, solution):
    prompt = f"""Provide constructive Recommendation on this solution for the student's issue based on their report:

    Issue: {issue}
    Solution: {solution}

    Recommendations:"""

    recommendation = recommendation_generator(
        prompt,
        max_length=300,  # Slightly longer for multiple recommendations
        do_sample=True,
        temperature=0.8,  # Match your feedback generator setting
        num_beams=4,  # Better quality generation
        repetition_penalty=1.5  # Prevent repetitive phrases
    )
    return recommendation[0]['generated_text']


Device set to use cpu


In [ ]:
df['Recommendation'] = [generate_recommendation(row['Reports'], row['Resolution'])
                       for _, row in tqdm(df.iterrows(), total=len(df), desc="Generating Recommendations")]



Generating Recommendations:   0%|          | 0/10 [00:00<?, ?it/s]

Generating Recommendations:  10%|█         | 1/10 [00:20<03:06, 20.72s/it]

Generating Recommendations:  20%|██        | 2/10 [00:40<02:42, 20.26s/it]

Generating Recommendations:  30%|███       | 3/10 [01:00<02:18, 19.85s/it]

Generating Recommendations:  40%|████      | 4/10 [01:20<02:00, 20.09s/it]

Generating Recommendations:  50%|█████     | 5/10 [01:50<01:57, 23.59s/it]

Generating Recommendations:  60%|██████    | 6/10 [02:29<01:55, 28.86s/it]

Generating Recommendations:  70%|███████   | 7/10 [02:48<01:16, 25.59s/it]

Generating Recommendations:  80%|████████  | 8/10 [03:08<00:48, 24.02s/it]

Generating Recommendations:  90%|█████████ | 9/10 [03:28<00:22, 22.68s/it]

Generating Recommendations: 100%|██████████| 10/10 [04:07<00:00, 24.79s/it]


In [ ]:
from IPython.display import display, HTML

def print_colored_report(df, index=0):
    issue = df["Reports"].iloc[index]
    solution = df["Resolution"].iloc[index]
    feedback = df["Feedback"].iloc[index]
    recommendation = df["Recommendation"].iloc[index]

    html_content = f"""
<div style="font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            max-width: 800px;
            margin: 20px auto;
            padding: 25px;
            border-radius: 10px;
            box-shadow: 0 4px 12px rgba(0,0,0,0.1);
            background: white;">

    <h1 style="text-align: center;
               color: #2c3e50;
               margin-bottom: 30px;
               border-bottom: 2px solid #f0f0f0;
               padding-bottom: 10px;">STUDENT REPORT</h1>

    <div style="margin-bottom: 25px;">
        <h2 style="color: #e74c3c;
                   border-left: 5px solid #e74c3c;
                   padding-left: 15px;
                   font-size: 18px;">REPORT</h2>
        <div style="background-color: #fdedec;
                   padding: 15px;
                   border-radius: 8px;
                   line-height: 1.7;
                   color: #7f8c8d;">{issue}</div>
    </div>

    <div style="margin-bottom: 25px;">
        <h2 style="color: #27ae60;
                   border-left: 5px solid #27ae60;
                   padding-left: 15px;
                   font-size: 18px;">SOLUTION</h2>
        <div style="background-color: #e8f8f5;
                   padding: 15px;
                   border-radius: 8px;
                   line-height: 1.7;
                   color: #7f8c8d;">{solution}</div>
    </div>

    <div style="margin-bottom: 25px;">
        <h2 style="color: #f39c12;
                   border-left: 5px solid #f39c12;
                   padding-left: 15px;
                   font-size: 18px;">FEEDBACK</h2>
        <div style="background-color: #fef9e7;
                   padding: 15px;
                   border-radius: 8px;
                   line-height: 1.7;
                   color: #7f8c8d;">{feedback}</div>
    </div>

    <div style="margin-bottom: 15px;">
        <h2 style="color: #9b59b6;
                   border-left: 5px solid #9b59b6;
                   padding-left: 15px;
                   font-size: 18px;">RECOMMENDATIONS</h2>
        <div style="background-color: #f5eef8;
                   padding: 15px;
                   border-radius: 8px;
                   line-height: 1.7;
                   color: #7f8c8d;">{recommendation}</div>
    </div>
</div>
"""
    display(HTML(html_content))


In [ ]:
print_colored_report(df,index=3)